In [1]:
!pip install PySastrawi

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import json
import pickle
from tensorflow.keras.callbacks import EarlyStopping
import nltk
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, callbacks, optimizers
import numpy as np

In [4]:
# Load the intents file
with open('nlp_dataset_faq.json') as data_file:
    intents = json.load(data_file)

In [5]:
# Download the 'punkt_tab' data package
nltk.download('punkt_tab')

# Pre-processing
words = []
classes = []
documents = []
ignore = ['','!', '"', "'", '(', ')', ',', '-', '.', ':', ';','[', ']', '_','?','.'
          'adalah', 'akan', 'aku', 'anda', 'atau', 'dalam', 'dan',
          'dari', 'dengan', 'di', 'dia', 'harus', 'ini', 'itu', 'jika', 'kami', 'kamu', 'ke',
          'kita', 'mereka', 'oleh', 'pada', 'saya', 'sebuah', 'sedang', 'sementara', 'tanpa',
          'tapi', 'telah', 'untuk', 'yang', '{', '}', 'merasa']



#tokenize
for intent in intents:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern.lower())  # Tokenizing
        words.extend([word for word in w if word not in ignore])  #Ignore
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
# Melakukan stemming dan normalisasi data
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# Menghapus class duplikat dengan 'set'
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

2052 documents
102 classes ['agama_kepercayaan_bot', 'apa_itu_gangguan_mental', 'badmood_dampak', 'badmood_efek_jangka_panjang', 'badmood_gejala', 'badmood_pengobatan', 'badmood_penyebab', 'badmood_preventif', 'badmood_solusi_pribadi', 'badmood_terhadap_kesehatan_fisik', 'badmood_terhadap_pekerjaan', 'baru_punya_pacar', 'bertengkar', 'bot_atau_orang_asli', 'bunuh_diri', 'cara_mengatasi_benci', 'cara_mengatasi_bingung', 'cara_mengatasi_cemas', 'cara_mengatasi_cemburu', 'cara_mengatasi_dendam', 'cara_mengatasi_diremehkan', 'cara_mengatasi_frustasi', 'cara_mengatasi_gangguan_mental', 'cara_mengatasi_hampa', 'cara_mengatasi_iri', 'cara_mengatasi_kecewa', 'cara_mengatasi_kesal', 'cara_mengatasi_malu', 'cara_mengatasi_marah', 'cara_mengatasi_merasa_sendiri', 'cara_mengatasi_overthinking', 'cara_mengatasi_rasa_bersalah', 'cara_mengatasi_sedih', 'cara_mengatasi_takut', 'cara_mengatasi_takut_gagal', 'cara_mengatasi_tidak_dihargai', 'cara_positive_thinking', 'diagnosis_gangguan_mental', 'dukunga

In [7]:
import random
import numpy as np

training = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    bag = [1 if w in pattern_words else 0 for w in words]

    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])


## MODEL

In [8]:
model = models.Sequential([
        layers.Input(shape=(len(train_x[0]),), name='input_layer'),
        layers.Dense(256, activation='relu', kernel_regularizer='l2', name='hidden_layer1'),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name='dropout1'),
        layers.Dense(128, activation='relu', kernel_regularizer='l2', name='hidden_layer2'),
        layers.BatchNormalization(),
        layers.Dropout(0.3, name='dropout2'),
        layers.Dense(len(train_y[0]), activation='softmax', name='output_layer')
    ])

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden_layer1 (Dense)                │ (None, 256)                 │         165,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout1 (Dropout)                   │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer2 (Dense)                │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout2 (Dropout)                   │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 102)                 │          13,158 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 213,478 (833.90 KB)

 Trainable params: 212,710 (830.90 KB)

 Non-trainable params: 768 (3.00 KB)

## K_FOLD

In [10]:
tf.compat.v1.reset_default_graph()

# Hyperparameters
num_folds = 6
batch_size = 8
learning_rate = 1e-4
epochs = 100


train_x = np.array(train_x)
train_y = np.array(train_y)


kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    print(f"\n=== Training Fold {fold + 1}/{num_folds} ===")
    
    X_train, X_val = train_x[train_index], train_x[val_index]
    y_train, y_val = train_y[train_index], train_y[val_index]

    tensorboard_callback = callbacks.TensorBoard(log_dir=f'./logs/fold_{fold+1}')
    early_stopping_callback = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[tensorboard_callback, early_stopping_callback],
        verbose=1
    )

    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} - Validation Accuracy: {val_accuracy:.4f}")
    fold_accuracies.append(val_accuracy)
    
avg_accuracy = np.mean(fold_accuracies)
print(f"\n=== Average Validation Accuracy: {avg_accuracy:.4f} ===")



=== Training Fold 1/6 ===
Epoch 1/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.0184 - loss: 10.2664 - val_accuracy: 0.0175 - val_loss: 8.8531
Epoch 2/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.0260 - loss: 8.8811 - val_accuracy: 0.0468 - val_loss: 8.1705
Epoch 3/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.0647 - loss: 8.1031 - val_accuracy: 0.1287 - val_loss: 7.5124
Epoch 4/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1065 - loss: 7.5518 - val_accuracy: 0.1901 - val_loss: 7.0419
Epoch 5/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1474 - loss: 7.1141 - val_accuracy: 0.2281 - val_loss: 6.6656
Epoch 6/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2103 - loss: 6.7188 - val_accuracy: 0.2807 - val_loss: 6.3375
Epoch 7/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2590 - loss: 6.3823 - val_accuracy: 0.3480 - val_loss: 6.0260
Epoch 8/100
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3

## Callbacks

In [11]:
early_stopping_callback = callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
tensorboard_callback = callbacks.TensorBoard(log_dir='./logs/full_training')

## Training tanpa split

In [12]:
from sklearn.utils import shuffle
train_x, train_y = shuffle(train_x, train_y, random_state=42)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train and save the final model
model.fit(train_x, train_y, epochs=100, batch_size=8, callbacks=[tensorboard_callback, early_stopping_callback], verbose=1)
model.save('final_model_trained_on_full_data.h5')

Epoch 1/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8473 - loss: 0.6759
Epoch 2/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8935 - loss: 0.5691
Epoch 3/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9140 - loss: 0.4894
Epoch 4/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9303 - loss: 0.4415
Epoch 5/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9401 - loss: 0.4059
Epoch 6/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9522 - loss: 0.3817
Epoch 7/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9611 - loss: 0.3523
Epoch 8/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9609 - loss: 0.3386
Epoch 9/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9625 - loss: 0.3256
Epoch 10/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9626 - loss: 0.3170
Epoch 11/100
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9694 - loss: 0.3203
Epoch 12/100
257/257 ━━━━━━━━━━━━━━━━━━━━

In [13]:
import json

# Save
try:
    with open("training_data.json", "w") as file:
        # Convert non-serializable data like NumPy arrays or other objects to lists
        data = {'words': words, 'classes': classes, 'train_x': train_x.tolist(), 'train_y': train_y.tolist()}
        json.dump(data, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [14]:
# Save
try:
    with open("training_data", "wb") as file:
        pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [15]:
# Load
try:
    with open("training_data", "rb") as file:
        data = pickle.load(file)

    words = data['words']
    classes = data['classes']
    train_x = np.array(data['train_x'])
    train_y = np.array(data['train_y'])
except IOError as e:
    print(f"Error loading training data: {e}")

# Load model
try:
    model = keras.models.load_model('final_model_trained_on_full_data.h5')
except IOError as e:
    print(f"Error loading model: {e}")


In [16]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
            if show_details:
                print(f"Found in bag: {w}")
    return bag

ERROR_THRESHOLD = 0.40



def classify(sentence):
    input_data = bow(sentence, words)
    results = model.predict(np.array([input_data]))[0]
    results = [[i, r] for i, r in enumerate(results) if r >= ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        for intent in intents:
            if intent['tag'] == results[0][0]:
                return random.choice(intent['responses'])
    return "Maaf, saya belum mengerti apa yang Anda bicarakan, perlu bantuan?."

In [18]:
print("0 to close")
while True:
    message = input("")
    if message == "0":
        break
    result = response(message)

    if result is not None and "~" in result:
        order = (result[1:])
        action(order)
    else:
        print(result)


0 to close


 saya mau mati saja


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Tolong jangan membuat keputusan tergesa-gesa. Kamu sangat berarti, dan aku ingin memastikan kamu mendapat dukungan yang tepat. Hubungi seseorang yang kamu percayai atau organisasi dukungan.


 saya lelah dengan capstone project dan Bangkit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Ketika merasa lelah, jangan ragu untuk berhenti sejenak dan mereset pikiranmu.


 saya merasa hampa


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Mencari dukungan sangat penting saat merasa seperti ini. Anda tidak perlu melalui ini sendirian.


 bagaimana cara mengatasi kehampaan?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Ingat bahwa rasa hampa adalah sementara, dan kamu bisa melewatinya.


 0


In [19]:
model = tf.keras.models.load_model('final_model_trained_on_full_data.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\dzikr\AppData\Local\Temp\tmpupbxtobw\assets


INFO:tensorflow:Assets written to: C:\Users\dzikr\AppData\Local\Temp\tmpupbxtobw\assets


Saved artifact at 'C:\Users\dzikr\AppData\Local\Temp\tmpupbxtobw'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 647), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 102), dtype=tf.float32, name=None)
Captures:
  2497633115152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633112272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633115728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633112656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633114384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633116688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633116496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633117264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633113232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2497633116880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  249763311553